# Protected area percentage in each cell
we compute the fraction of area covered by protected areas inside each pixel 

In [2]:
import json
import requests
from pprint import pprint
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import math
import shapely.wkb 
from shapely.ops import cascaded_union

%matplotlib inline

### Protected areas

**Strict Reserves**

```sql
SELECT the_geom, the_geom_webmercator FROM wdpa_protected_areas
WHERE iucn_cat = 'Ia' OR iucn_cat = 'Ib' OR iucn_cat = 'II' OR iucn_cat = 'III'
```

**Community-based Conservation Areas**

```sql
SELECT the_geom, the_geom_webmercator FROM community_conservation_areas
```

**Biosphere Reserves:**

```sql
SELECT the_geom, the_geom_webmercator FROM wdpa_protected_areas 
WHERE desig_eng like '%Biosphere%'
```

**Other**

```sql
SELECT * FROM wdpa_protected_areas 
WHERE  iucn_cat != 'Ia' AND iucn_cat !='Ib' AND iucn_cat !='II' AND iucn_cat !='III'
AND desig_eng not like '%Aborig%' AND desig_eng not like '%Indigen%' 
AND desig_eng not like '%Commun%'AND desig_eng not like '%Conservanc%' AND desig_eng not like '%Local%'
AND desig_eng not like '%Region%' AND desig_eng not like '%Trust%' AND desig_eng not like '%Conservator%' 
AND desig_eng not like '%Private%' AND desig_eng not like '%Nature Center%'
```

**Read Global Facets table**

In [3]:
df = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/Half-Earth/work/data/global_facets_attr_pressures_all/global_facets_attr_pressures.shp')

In [4]:
df.columns = map(str.lower, df.columns)
df.head()

,cell_id,taxa,sr,ave_rsr,sum_rsr,focal_spp,ave_rsr_pc,sum_rsr_pc,sr_pc,isisland,prop_land,urban,agricultur,rainfed,geometry
0,26.25--112.25,fishes,2989.0,0.000256,0.763941,"Hypomesus_olidus,Pholis_laeta,Mylocheilus_caur...",98.213758,99.774352,99.830067,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-112 26, -112.25 26.0002148979084, -..."
1,9.25-124.75,fishes,2989.0,0.000236,0.704408,"Seriphus_politus,Typhlogobius_californiensis,C...",97.394739,99.570990,99.830067,0.0,0.0,0.0,0.0,0.0,"POLYGON ((125 9, 124.5 9, 124.5 9.5, 125 9.5, ..."
2,33.75-133.75,fishes,2987.0,0.000220,0.656372,"Pholis_laeta,Mylocheilus_caurinus,Liparis_call...",96.682694,99.329184,99.828396,0.0,0.0,0.0,0.0,0.0,"POLYGON ((134 33.5, 133.75 33.5002510300208, 1..."
3,-1.75-134.25,fishes,2987.0,0.000223,0.665440,"Ambassis_jacksoniensis,Vanacampus_margaritifer...",96.815297,99.377100,99.828396,0.0,0.0,0.0,0.0,0.0,"POLYGON ((134.5 -2, 134 -2, 134 -1.5, 134.5 -1..."
4,-17.25-178.25,fishes,2987.0,0.000230,0.686238,"Sprattus_muelleri,Colistium_guntheri,Trachinop...",97.118390,99.484074,99.828396,0.0,0.0,0.0,0.0,0.0,"POLYGON ((178.5 -17.5, 178.25 -17.500156419814..."


## Protected areas percentage

### Terrestrial grid

In [5]:
query =("SELECT cartodb_id, cell_id, the_geom FROM terrestrial_grid")

account = 'half-earth'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

sql = {"q": query}
r = requests.get(urlCarto, params=sql)

data = r.json()

terrestrial = gpd.GeoDataFrame(data.get("rows"))
# Change geometry from WKB to WKT format
terrestrial['geometry'] = terrestrial.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
terrestrial.drop(columns='the_geom', inplace=True)
terrestrial.sort_values(by=['cartodb_id'], inplace=True)
terrestrial.head()

,cartodb_id,cell_id,geometry
17,1,100,(POLYGON ((-80.00000025386289 77.3465165253908...
18,2,1000,"(POLYGON ((99.99999928535659 70.5845799262466,..."
19,3,10000,"(POLYGON ((99.99999928535659 31.2274296654427,..."
20,4,10001,"(POLYGON ((100.999999282797 31.2274296654427, ..."
21,5,10002,"(POLYGON ((101.999999280237 31.2274296654427, ..."


In [6]:
print('Number of cells:', len(terrestrial))

Number of cells: 12670


### Marine grid 

In [7]:
query =("SELECT cartodb_id, cell_id, the_geom FROM marine_grid")

account = 'half-earth'
urlCarto = f"https://{account}.carto.com/api/v2/sql"

sql = {"q": query}
r = requests.get(urlCarto, params=sql)

data = r.json()

marine = gpd.GeoDataFrame(data.get("rows"))
# Change geometry from WKB to WKT format
marine['geometry'] = marine.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
marine.drop(columns='the_geom', inplace=True)
marine.sort_values(by=['cartodb_id'], inplace=True)
marine.head()

,cartodb_id,cell_id,geometry
295,1,6.25-121.75,"(POLYGON ((122 6, 121.5 6, 121.5 6.5, 122 6.5,..."
296,2,7.25-151.75,"(POLYGON ((152 7, 151.5 7, 151.5 7.49999999999..."
297,3,41.75-141.25,"(POLYGON ((141.5 41.5, 141.25 41.5002706759897..."
298,4,-4.25-55.75,"(POLYGON ((56 -4.5, 55.5 -4.5, 55.5 -4, 56 -4,..."
299,5,-9.75-141.25,"(POLYGON ((141.5 -9.999999999999998, 141.25 -1..."


In [8]:
print('Number of cells:', len(marine))

Number of cells: 179484


**Strict Reserves**

```sql
WITH a as (SELECT the_geom FROM wdpa_protected_areas
           WHERE iucn_cat = 'Ia' OR iucn_cat = 'Ib' OR iucn_cat = 'II' OR iucn_cat = 'III')
           
SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id
FROM terrestrial_grid as b, a 
WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > 0 AND b.cartodb_id <= 4000
GROUP BY b.cell_id, b.cartodb_id
```

Terrestrial

In [9]:
strict = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 500
for i in range(math.ceil(len(terrestrial)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT the_geom FROM wdpa_protected_areas" 
            " WHERE iucn_cat = 'Ia' OR iucn_cat = 'Ib' OR iucn_cat = 'II' OR iucn_cat = 'III')"
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM terrestrial_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM terrestrial_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    strict = pd.concat([strict,area])

Marine

In [10]:
strict_marine = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 2000
for i in range(math.ceil(len(marine)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT the_geom FROM wdpa_protected_areas" 
            " WHERE iucn_cat = 'Ia' OR iucn_cat = 'Ib' OR iucn_cat = 'II' OR iucn_cat = 'III')"
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM marine_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM marine_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    strict_marine = pd.concat([strict_marine,area])

In [20]:
strict = strict[['cell_id','percentage']]
strict.rename(columns={"percentage": "strict_percentage"}, inplace=True)
strict_marine = strict_marine[['cell_id','percentage']]
strict_marine.rename(columns={"percentage": "strict_percentage"}, inplace=True)
strict_all = pd.concat([strict,strict_marine])
len(strict_all)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


**Community-based Conservation Areas**

```sql
WITH a as (SELECT the_geom FROM community_conservation_areas)
           
SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id
FROM terrestrial_grid as b, a 
WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > 0 AND b.cartodb_id <= 4000
GROUP BY b.cell_id, b.cartodb_id
```

Terrestrial

In [12]:
conservation = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 500
for i in range(math.ceil(len(terrestrial)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT the_geom FROM community_conservation_areas)" 
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM terrestrial_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM terrestrial_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    conservation = pd.concat([conservation,area])

Marine

In [13]:
conservation_marine = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 2000
for i in range(math.ceil(len(marine)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT the_geom FROM community_conservation_areas)" 
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM marine_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM marine_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    conservation_marine = pd.concat([conservation_marine,area])

In [29]:
conservation = conservation[['cell_id','percentage']]
conservation.rename(columns={"percentage": "conservation_percentage"}, inplace=True)
conservation_marine = conservation_marine[['cell_id','percentage']]
conservation_marine.rename(columns={"percentage": "strict_percentage"}, inplace=True)
conservation_all = pd.concat([conservation,conservation_marine])
len(conservation_all)

192154

**Biosphere Reserves:**

```sql
WITH a as (SELECT the_geom, the_geom_webmercator FROM wdpa_protected_areas 
           WHERE desig_eng like '%Biosphere%')
           
SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id
FROM terrestrial_grid as b, a 
WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > 0 AND b.cartodb_id <= 4000
GROUP BY b.cell_id, b.cartodb_id
```

Terrestrial

In [14]:
biosphere = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 500
for i in range(math.ceil(len(terrestrial)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT the_geom, the_geom_webmercator FROM wdpa_protected_areas" 
            " WHERE desig_eng like '%Biosphere%')" 
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM terrestrial_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM terrestrial_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    biosphere = pd.concat([biosphere,area])

Marine

In [15]:
biosphere_marine = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 2000
for i in range(math.ceil(len(marine)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT the_geom, the_geom_webmercator FROM wdpa_protected_areas" 
            " WHERE desig_eng like '%Biosphere%')" 
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM marine_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM marine_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    biosphere_marine = pd.concat([biosphere_marine,area])

In [30]:
biosphere = biosphere[['cell_id','percentage']]
biosphere.rename(columns={"percentage": "biosphere_percentage"}, inplace=True)
biosphere_marine = biosphere_marine[['cell_id','percentage']]
biosphere_marine.rename(columns={"percentage": "strict_percentage"}, inplace=True)
biosphere_all = pd.concat([biosphere,biosphere_marine])
len(biosphere_all)

192154

**Other**

```sql
WITH a as (SELECT * FROM wdpa_protected_areas 
           WHERE  iucn_cat != 'Ia' AND iucn_cat !='Ib' AND iucn_cat !='II' AND iucn_cat !='III'
           AND desig_eng not like '%Aborig%' AND desig_eng not like '%Indigen%' 
           AND desig_eng not like '%Commun%'AND desig_eng not like '%Conservanc%' AND desig_eng not like '%Local%'
           AND desig_eng not like '%Region%' AND desig_eng not like '%Trust%' AND desig_eng not like '%Conservator%' 
           AND desig_eng not like '%Private%' AND desig_eng not like '%Nature Center%')
           
SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id
FROM terrestrial_grid as b, a 
WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > 0 AND b.cartodb_id <= 4000
GROUP BY b.cell_id, b.cartodb_id
```

Terrestrial

In [16]:
other = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 500
for i in range(math.ceil(len(terrestrial)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT * FROM wdpa_protected_areas" 
            " WHERE  iucn_cat != 'Ia' AND iucn_cat !='Ib' AND iucn_cat !='II' AND iucn_cat !='III'"
            " AND desig_eng not like '%Aborig%' AND desig_eng not like '%Indigen%'" 
            " AND desig_eng not like '%Commun%'AND desig_eng not like '%Conservanc%' AND desig_eng not like '%Local%'"
            " AND desig_eng not like '%Region%' AND desig_eng not like '%Trust%' AND desig_eng not like '%Conservator%'" 
            " AND desig_eng not like '%Private%' AND desig_eng not like '%Nature Center%')" 
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM terrestrial_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM terrestrial_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    other = pd.concat([other,area])

Marine

In [17]:
other_marine = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'tot_area', 'area', 'percentage'])

step = 2000
for i in range(math.ceil(len(marine)/step)):
    #print(0+(step*i), step*(i+1))

    min_id = 0+(step*i)
    max_id = step*(i+1)

    account = 'half-earth'
    urlCarto = f"https://{account}.carto.com/api/v2/sql"

    # we compute the area covered by protected areas inside each cell
    query =("WITH a as (SELECT * FROM wdpa_protected_areas" 
            " WHERE  iucn_cat != 'Ia' AND iucn_cat !='Ib' AND iucn_cat !='II' AND iucn_cat !='III'"
            " AND desig_eng not like '%Aborig%' AND desig_eng not like '%Indigen%'" 
            " AND desig_eng not like '%Commun%'AND desig_eng not like '%Conservanc%' AND desig_eng not like '%Local%'"
            " AND desig_eng not like '%Region%' AND desig_eng not like '%Trust%' AND desig_eng not like '%Conservator%'" 
            " AND desig_eng not like '%Private%' AND desig_eng not like '%Nature Center%')" 
            " SELECT sum(ST_Area(ST_Intersection(b.the_geom, a.the_geom))) as area, b.cell_id, b.cartodb_id"
            " FROM marine_grid as b, a" 
           f" WHERE ST_Intersects(b.the_geom, a.the_geom) AND b.cartodb_id > {min_id} AND b.cartodb_id <= {max_id}"
            " GROUP BY b.cell_id, b.cartodb_id")
    
    sql = {"q": query}
    
    r = requests.get(urlCarto, params=sql)
    data = r.json()
    
    #print(r.status_code)
    #if r.status_code == 400:
    #    print('Error: smaller step is required')
    #    break
        
    area = gpd.GeoDataFrame(data.get("rows"))
    if area.empty:
        area = pd.DataFrame(columns = ['cartodb_id', 'cell_id', 'area'])
    else:
        area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we compute the area inside each cell
    query =("SELECT ST_Area(the_geom) as tot_area, cell_id, cartodb_id"
            " FROM marine_grid" 
           f" WHERE cartodb_id > {min_id} AND cartodb_id <= {max_id}")
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    tot_area = gpd.GeoDataFrame(data.get("rows"))
    tot_area.sort_values(by=['cartodb_id'], inplace=True)
    
    # we merge the tables
    area = pd.merge(tot_area, area, how='left', on=['cartodb_id', 'cell_id'])
    
    # we compute the percentage
    area['area'].fillna(0, inplace=True)
    area['percentage'] = area.apply(lambda x: x['area']/x['tot_area']*100, axis=1)
    
    other_marine = pd.concat([other_marine,area])

In [31]:
other = other[['cell_id','percentage']]
other.rename(columns={"percentage": "other_percentage"}, inplace=True)
other_marine = other_marine[['cell_id','percentage']]
other_marine.rename(columns={"percentage": "strict_percentage"}, inplace=True)
other_all = pd.concat([other,other_marine])
len(other_all)

192154

**Add percentage columns to table**

In [35]:
strict_all.rename(columns={"strict_percentage": "strict"}, inplace=True)
conservation_all.rename(columns={"strict_percentage": "conservation"}, inplace=True)
biosphere_all.rename(columns={"strict_percentage": "biosphere"}, inplace=True)
other_all.rename(columns={"strict_percentage": "other"}, inplace=True)

In [50]:
df_new = pd.merge(df, strict_all, how='left', on=['cell_id'])
df_new = pd.merge(df_new, conservation_all, how='left', on=['cell_id'])
df_new = pd.merge(df_new, biosphere_all.head(), how='left', on=['cell_id'])
df_new = pd.merge(df_new, other_all, how='left', on=['cell_id'])

In [51]:
df_new.head()

,cell_id,taxa,sr,ave_rsr,sum_rsr,focal_spp,ave_rsr_pc,sum_rsr_pc,sr_pc,isisland,prop_land,urban,agricultur,rainfed,geometry,strict,conservation,biosphere,other
0,26.25--112.25,fishes,2989.0,0.000256,0.763941,"Hypomesus_olidus,Pholis_laeta,Mylocheilus_caur...",98.213758,99.774352,99.830067,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-112 26, -112.25 26.0002148979084, -...",0.0,0.000000,NaN,0.0
1,9.25-124.75,fishes,2989.0,0.000236,0.704408,"Seriphus_politus,Typhlogobius_californiensis,C...",97.394739,99.570990,99.830067,0.0,0.0,0.0,0.0,0.0,"POLYGON ((125 9, 124.5 9, 124.5 9.5, 125 9.5, ...",0.0,0.000000,NaN,0.0
2,33.75-133.75,fishes,2987.0,0.000220,0.656372,"Pholis_laeta,Mylocheilus_caurinus,Liparis_call...",96.682694,99.329184,99.828396,0.0,0.0,0.0,0.0,0.0,"POLYGON ((134 33.5, 133.75 33.5002510300208, 1...",0.0,0.000000,NaN,0.0
3,-1.75-134.25,fishes,2987.0,0.000223,0.665440,"Ambassis_jacksoniensis,Vanacampus_margaritifer...",96.815297,99.377100,99.828396,0.0,0.0,0.0,0.0,0.0,"POLYGON ((134.5 -2, 134 -2, 134 -1.5, 134.5 -1...",0.0,0.000000,NaN,0.0
4,-17.25-178.25,fishes,2987.0,0.000230,0.686238,"Sprattus_muelleri,Colistium_guntheri,Trachinop...",97.118390,99.484074,99.828396,0.0,0.0,0.0,0.0,0.0,"POLYGON ((178.5 -17.5, 178.25 -17.500156419814...",0.0,51.701605,NaN,0.0


## Add rank columns to table

In [57]:
df_new['taxa'].unique()

array(['fishes', 'birds', 'mammals', 'amphibians', 'cacti', 'conifers',
       'turtles', 'all'], dtype=object)

In [59]:
taxa_array = ['fishes', 'birds', 'mammals', 'amphibians', 'cacti', 'conifers',
       'turtles']

In [60]:
rank_all = pd.DataFrame(columns = ['cell_id', 'rank_sr', 'rank_rsr', 'rank_total', 'taxa'])
for taxa in taxa_array:
    print(taxa)
    # Rank rs and ave_rsr (mammals)
    api_key = '742c956aae8c74b51b6c554733a3b440fb81cbfd'
     
    sql = f"SELECT cell_id, sr, ave_rsr FROM global_facets_attr_pressures where taxa = '{taxa}' order by sr  DESC"
    url = f"https://carto.mol.org/user/half-earth/api/v2/sql?api_key={api_key}&q={sql}"
    r = requests.get(f"{url}")
    if r.status_code == 200:
        responses = (r.json())
    else:
        print(f"Error: {r.status_code}")
        
    rank = pd.DataFrame(responses.get("rows"))
    
    sr_max = rank['sr'].max()
    sr_min = rank['sr'].min()
    rsr_max = rank['ave_rsr'].max()
    rsr_min = rank['ave_rsr'].min()
    
    rank['rank_sr'] = rank.apply(lambda x: (x['sr']-sr_min)/(sr_max-sr_min), axis=1)
    rank['rank_rsr'] = rank.apply(lambda x: (x['ave_rsr']-rsr_min)/(rsr_max-rsr_min), axis=1)
    rank['rank_total'] = rank.apply(lambda x: x['rank_sr'] + x['rank_rsr'], axis=1)
    
    rank = rank[['cell_id', 'rank_sr', 'rank_rsr', 'rank_total']]
    rank['taxa'] = taxa
    
    rank_all = pd.concat([rank_all,rank])

fishes
birds
mammals
amphibians
cacti
conifers
turtles


In [62]:
df_new = pd.merge(df_new, rank_all, how='left', on=['cell_id', 'taxa'])

In [65]:
df_new.head()

,cell_id,taxa,sr,ave_rsr,sum_rsr,focal_spp,ave_rsr_pc,sum_rsr_pc,sr_pc,isisland,...,agricultur,rainfed,geometry,strict,conservation,biosphere,other,rank_sr,rank_rsr,rank_total
0,26.25--112.25,fishes,2989.0,0.000256,0.763941,"Hypomesus_olidus,Pholis_laeta,Mylocheilus_caur...",98.213758,99.774352,99.830067,0.0,...,0.0,0.0,"POLYGON ((-112 26, -112.25 26.0002148979084, -...",0.0,0.000000,NaN,0.0,0.861592,0.037323,0.898915
1,9.25-124.75,fishes,2989.0,0.000236,0.704408,"Seriphus_politus,Typhlogobius_californiensis,C...",97.394739,99.570990,99.830067,0.0,...,0.0,0.0,"POLYGON ((125 9, 124.5 9, 124.5 9.5, 125 9.5, ...",0.0,0.000000,NaN,0.0,0.861592,0.034412,0.896003
2,33.75-133.75,fishes,2987.0,0.000220,0.656372,"Pholis_laeta,Mylocheilus_caurinus,Liparis_call...",96.682694,99.329184,99.828396,0.0,...,0.0,0.0,"POLYGON ((134 33.5, 133.75 33.5002510300208, 1...",0.0,0.000000,NaN,0.0,0.861015,0.032084,0.893099
3,-1.75-134.25,fishes,2987.0,0.000223,0.665440,"Ambassis_jacksoniensis,Vanacampus_margaritifer...",96.815297,99.377100,99.828396,0.0,...,0.0,0.0,"POLYGON ((134.5 -2, 134 -2, 134 -1.5, 134.5 -1...",0.0,0.000000,NaN,0.0,0.861015,0.032528,0.893543
4,-17.25-178.25,fishes,2987.0,0.000230,0.686238,"Sprattus_muelleri,Colistium_guntheri,Trachinop...",97.118390,99.484074,99.828396,0.0,...,0.0,0.0,"POLYGON ((178.5 -17.5, 178.25 -17.500156419814...",0.0,51.701605,NaN,0.0,0.861015,0.033545,0.894560


Save table

In [ ]:
df_new.to_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/Half-Earth/work/data/global_facets_attr_pressures_iker/global_facets_attr_pressures_iker.shp')